In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

import lightgbm as lgb
import xgboost as xgb

import matplotlib.pyplot as plt
from datetime import datetime

%matplotlib inline

In [2]:
colnames = [
    "TV_TYPE",          # Тип тарелки                   (Input)
    "ls_num",           # Номер лицевого счета          (ID)
    "MON_ACTIVATION",   # Месяц подключения             (Constant)     
    "CUST_ID",          # Номер клиента                 (ID)
    "ABONID",           # Номер абонента                (ID)
    "smart_card_num",   # Номер смарт-карты             (ID, Unique x18)
    "CHIP_ID",          # Номер чипа                    (ID, almost unique x18)
    "PER",              # Месяц наблюдения              (= PERIOD_, redundant)
    "_1M",              # Отток на горизонте 1 месяц    (Target)
    "_3M",              # Отток на горизонте 3 месяца   (Target)
    "_6M",              # Отток на горизонте 6 месяца   (Target)
    "TOTAL",            # Флаг расторжения контракта    (Target)
    "BASE_PACK",        # Тарифный план текущий         (Input)
    "BASE_PACK_FIRST",  # Тарифный план первый          (Input)
    "PERIOD_",          # Месяц наблюдения              (Input)
    "CURRENT_TARPLANID" # ID тарифного плана            (ID)
]

# Данные помесячного присоединения клиентов+

In [3]:
df_stv = pd.read_csv(r"./STV/....txt", encoding="utf-8", sep="\t")

D:\Programs\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df_stv.head()

,TV_TYPE,ls_num,MON_ACTIVATION,CUST_ID,ABONID,smart_card_num,CHIP_ID,PER,_1M,_3M,_6M,TOTAL,BASE_PACK,BASE_PACK_FIRST,PERIOD_,CURRENT_TARPLANID
0,ИТВ,8970101006118013776,12/1/2018,135388559,172269170,79175274868,7700041856,DEC19,0,1,1,1,СТВ. Пакет телеканалов: Базовый,СТВ. Пакет телеканалов: Базовый,12/1/2019,80938
1,ИТВ,8970101005037856640,12/1/2018,155083390,192939344,79153213516,7700013091,DEC18,1,1,1,1,СТВ. Пакет телеканалов: Базовый,СТВ. Пакет телеканалов: Базовый,12/1/2018,80938
2,ИТВ,8970101006118009198,12/1/2018,179929195,219811084,79169926437,NaN,APR19,0,0,1,1,СТВ. Пакет телеканалов: Базовый,СТВ. Пакет телеканалов: Базовый,4/1/2019,80938
3,ИТВ,8970101006118009723,12/1/2018,117572446,153852051,79169985926,7700023486,OCT19,0,0,0,0,СТВ. Пакет телеканалов: Базовый на год,СТВ. Пакет телеканалов: Базовый на год,10/1/2019,80959
4,ИТВ,8970101006118004253,12/1/2018,113290717,149390073,79169538499,7700021300,JAN19,1,1,1,1,СТВ. Пакет телеканалов: Базовый на год,СТВ. Пакет телеканалов: Базовый на год,1/1/2019,80959


In [5]:
from sklearn.preprocessing import OneHotEncoder

def transform_sample(df, index=True, encoding=False, transformer=None):
    select_cols = ['BASE_PACK', 'BASE_PACK_FIRST', 'day_space', '_1M', 'TV_TYPE'] 
    if index:
        select_cols.append('smart_card_num')
    
    df = df.copy()
    
    BASE_PACK_NAN = 'Базовый на год (СТВ) (DVB-S) (МАСС) (082016)'
    BASE_PACK_FIRST_NAN = 'Расширенный Плюс. Полгода за полцены (DVB-S) (МАСС)'

    df['BASE_PACK'].replace({'?': BASE_PACK_NAN}, inplace=True)
    df['BASE_PACK_FIRST'].replace({'?': BASE_PACK_FIRST_NAN}, inplace=True)

    df['MON_ACTIVATION'] = df['MON_ACTIVATION'].apply(lambda x: datetime.strptime(x, "%m/%d/%Y"))
    df['PERIOD_'] = df['PERIOD_'].apply(lambda x: datetime.strptime(x, "%m/%d/%Y"))
    df['day_space'] = (df['PERIOD_'] - df['MON_ACTIVATION']).dt.days
    
    if encoding:
        col = ['BASE_PACK', 'BASE_PACK_FIRST']
        if transformer is None:
            t = OneHotEncoder(sparse=False, handle_unknown='ignore')
            values = t.fit_transform(df[col])
        else:
            t = transformer
            values = t.transform(df[col])
        new_cols = t.get_feature_names().tolist()
            
        tmp = pd.DataFrame(values, columns=new_cols)
        df = pd.concat([df, tmp], axis=1).drop(col, axis=1)
        
        for c in col:
            select_cols.remove(c)
        select_cols.extend(new_cols)
        
        return df[select_cols], t
        
    else:
        for col in ['TV_TYPE', 'BASE_PACK', 'BASE_PACK_FIRST']:
            df[col] = df[col].astype('category')
        return df[select_cols]


#     CHIP_ID_NAN = 0
#     df["CHIP_ID"].replace({'?': CHIP_ID_NAN}, inplace=True)
#     df["CHIP_ID"].fillna(CHIP_ID_NAN, inplace=True)

In [6]:
data = transform_sample(df_stv, index=True)
data.head()

,BASE_PACK,BASE_PACK_FIRST,day_space,_1M,TV_TYPE,smart_card_num
0,СТВ. Пакет телеканалов: Базовый,СТВ. Пакет телеканалов: Базовый,365,0,ИТВ,79175274868
1,СТВ. Пакет телеканалов: Базовый,СТВ. Пакет телеканалов: Базовый,0,1,ИТВ,79153213516
2,СТВ. Пакет телеканалов: Базовый,СТВ. Пакет телеканалов: Базовый,121,0,ИТВ,79169926437
3,СТВ. Пакет телеканалов: Базовый на год,СТВ. Пакет телеканалов: Базовый на год,304,0,ИТВ,79169985926
4,СТВ. Пакет телеканалов: Базовый на год,СТВ. Пакет телеканалов: Базовый на год,31,1,ИТВ,79169538499


In [7]:
def split_by_index(df, index='smart_card_num', target='_1M', test_size=0.30, random_state=0):
    ids = df[index].unique()
    ids_train, ids_test = train_test_split(ids, test_size=test_size, random_state=random_state)

    train = df[df[index].isin(ids_train)]
    test = df[df[index].isin(ids_test)]

    X_train = train.drop([index, target], axis=1)
    X_test = test.drop([index, target], axis=1)

    y_train = train[target]
    y_test = test[target]
    
    return X_train, X_test, y_train, y_test

In [8]:
X_train, X_test, y_train, y_test = split_by_index(data)
# X_train, X_test, y_train, y_test = train_test_split(data.drop(['_1M', 'smart_card_num'], 1), data['_1M'], test_size=0.30, random_state=0)

In [10]:
%%time
model = lgb.LGBMClassifier(n_jobs=8, random_state=0)
model.fit(X_train, y_train)

Wall time: 1.15 s


LGBMClassifier(n_jobs=8, random_state=0)

In [11]:
y_test_pred = model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_test_pred)
roc_auc

0.7072749048803264

In [12]:
df_oot = pd.read_csv(r"./STV/STV_base_2019_01.txt", encoding="utf-8", sep="\t")
df_oot = transform_sample(df_oot, index=False)

D:\Programs\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (3,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [13]:
X_oot = df_oot.drop('_1M', axis=1)
y_oot = df_oot['_1M']

y_oot_pred = model.predict_proba(X_oot)[:, 1]
roc_auc = roc_auc_score(y_oot, y_oot_pred)
roc_auc

0.7115116968696127

In [14]:
"""
TV_TYPE - незначимый признак
"""

'\nTV_TYPE - незначимый признак\n'

In [16]:
# Загрузим данные
df = pd.read_csv(r"./STV/....txt", encoding="utf-8", sep="\t")

D:\Programs\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [17]:
# Обработаем так же, как делали раньше
BASE_PACK_NAN = 'Базовый на год (СТВ) (DVB-S) (МАСС) (082016)'
BASE_PACK_FIRST_NAN = 'Расширенный Плюс. Полгода за полцены (DVB-S) (МАСС)'

df['BASE_PACK'].replace({'?': BASE_PACK_NAN}, inplace=True)
df['BASE_PACK_FIRST'].replace({'?': BASE_PACK_FIRST_NAN}, inplace=True)

df['MON_ACTIVATION'] = df['MON_ACTIVATION'].apply(lambda x: datetime.strptime(x, "%m/%d/%Y"))
df['PERIOD_'] = df['PERIOD_'].apply(lambda x: datetime.strptime(x, "%m/%d/%Y"))
df['day_space'] = (df['PERIOD_'] - df['MON_ACTIVATION']).dt.days

In [18]:
# Важно! Преобразовываем категориальные поля в тип "category".
# По сути, это и есть все отличие от других алгоритмов
# После преобразования в данный тип можно подавать данные в модель
for col in ['TV_TYPE', 'BASE_PACK', 'BASE_PACK_FIRST']:
    df[col] = df[col].astype('category')

In [19]:
# Оставим только необходимые поля
df_filt = df[['TV_TYPE', 'BASE_PACK', 'BASE_PACK_FIRST', 'day_space', '_1M', 'smart_card_num']]

In [20]:
# split_by_index - это самописная функция, разделяющая датафрейм по индексу 'smart_card_num', её определение находится выше
X_train, X_test, y_train, y_test = split_by_index(df_filt)

In [21]:
# Объявляем и обучаем модель
# Параметр n_jobs отвечает за распараллеливание вычислений на несколько ядер процессора
model = lgb.LGBMClassifier(n_jobs=4)
model.fit(X_train, y_train)

LGBMClassifier(n_jobs=4)

In [22]:
y_test_pred = model.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, y_test_pred)

0.7071495615007266

In [ ]:
"""
Довольно часто в используют не ROC AUC для качества модели, а метрику Gini.
Она вычисляется почти также, только это отношение верхнего треугольника roc-кривой к идеальному
(объяснение сложное, можно забить и просто посмотреть на формулу)

Gini = 2*roc_auc - 1.
Смысл в том, что это тот же "рок аук", только измеряется Gini в интервале [0; 1], а не [0.5; 1], 
   и позволяет выражать метрику качества в процентах.

Для нашей текущей модели roc_auc = 0.7, значит gini = 2*roc_auc - 1 = 0.4 = 40%.
Её удобнее интерпретировать - нулевой gini значит, что модель рандомная, а 100% - что модель идеально ранжирует.

Тем не менее, надо понимать, что Gini = 40% не означает, что точность попадания модели 40%, 
   это просто некоторая мера ранжирования модели.
"""

In [25]:
def gini(y_true, y_pred):
    return 2*roc_auc_score(y_true, y_pred) - 1

In [26]:
gini(y_test, y_test_pred)

0.4142991230014532